In [1]:
import numpy as np
import tensorflow as tf
import os

import librosa
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

opj = os.path.join

In [2]:
gtzan_path = r'datasets\gtzan'
genres_dir = opj(gtzan_path, 'genres_original')

In [3]:
def load_data_by_classes(dataset_path):
    data = []
    labels = []
    classes = os.listdir(dataset_path)
    for cl in tqdm(classes):
        full_p = opj(dataset_path, cl)
        for file in os.listdir(full_p):
            for i in range(0, 20, 2):
                filepath = os.path.join(full_p, file)  
                try:
                    wavedata, _ = librosa.load(filepath, sr=None, mono=True, offset=i, duration=2)
                except:
                    continue
                wavedata = wavedata[:, np.newaxis]
                data.append(wavedata)
                labels.append(cl)

    return data, labels

In [4]:
data, labels = load_data_by_classes(genres_dir)

 50%|█████████████████████████████████████████▌                                         | 5/10 [00:01<00:01,  2.76it/s]C:\ProgramData\Anaconda3\envs\deeplearning\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.90it/s]


In [5]:
sizes = [len(item) for item in data]
min_size = min(sizes)

data = [item[:min_size] for item in data]

# int(min_size/16)

In [6]:
data = np.array(data)
# data = data[..., np.newaxis]


le = LabelEncoder()
labels = le.fit_transform(labels)
labels_1hot = to_categorical(labels)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.15)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [8]:
X_train = tf.convert_to_tensor(X_train)
X_val = tf.convert_to_tensor(X_val)
X_test = tf.convert_to_tensor(X_test)

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [9]:
from tensorflow.keras.layers import Conv1D, Dropout, LSTM, TimeDistributed, Activation, Dense, Input, MaxPooling1D, Lambda, Flatten, BatchNormalization, LSTM

In [10]:
N_LAYERS = 4
CONV_FILTER_COUNT = 64
FILTER_LENGTH = 25

GENRES = 10

In [11]:
import tensorflow.keras.backend as K

from tensorflow.keras import Model, Sequential

from tensorflow.keras.optimizers import RMSprop

In [12]:
LSTM_COUNT = 256
BATCH_SIZE = 32
EPOCH_COUNT = 80

kernel_size = 100

In [13]:
model = Sequential()

model.add(LSTM(units=128, dropout=0.05, return_sequences=True, input_shape=data.shape[1:]))
model.add(LSTM(units=32,  dropout=0.05, return_sequences=False))
model.add(Dense(units=10, activation="softmax"))

model.compile(loss='categorical_crossentropy' , optimizer=RMSprop(learning_rate=0.0001) , metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = 24, batch_size=8)

Epoch 1/24
296/849 [=========>....................] - ETA: 16:34 - loss: 2.3022 - accuracy: 0.1022

InternalError:    Failed to call ThenRnnBackward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 2, 0, 0 , [num_layers, input_size, num_units, dir_count, max_seq_length, batch_size, cell_num_units]: [1, 128, 32, 1, 44100, 8, 32] 
	 [[{{node gradients/CudnnRNN_grad/CudnnRNNBackprop}}]]
	 [[PartitionedCall]] [Op:__inference_train_function_4997]

Function call stack:
train_function -> train_function -> train_function
